In [1]:
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio as rio
import pandas as pd
import glob
import pickle
import numpy as np
from csslconstants import *
from cmcrameri import cm

homedir = '/home/marianne/Documents/cssltimeseries/'
datadir = homedir + 'data/' 
os.chdir(homedir)

## load data from the prep_cssl_data.py process
with open(datadir + 'cssldata.pickle', 'rb') as handle:
    cssl = pickle.load(handle)


In [2]:
## get CSSL  monthly data
csslmonthly = pd.read_csv(datadir + 'csslmonthly.csv')

In [3]:
csslmonthly

,Date,Air Temp Max (C),Air Temp Min (C),24-hour Total Precip (mm),Season Total Precip (mm),% of Precip as Snow,% of Precip as Rain,New Snow (cm),Season Total Snow (cm),Snowpack depth (cm),Snow Water Equivalent (cm),dt
0,1970-10-31,13.176774,-0.179032,2.580968,25.899806,NaN,NaN,1.651000,14.584516,7.292258,0.000000,2.488320e+16
1,1970-11-30,5.938500,-2.053500,15.820571,273.658067,NaN,NaN,5.533571,79.048667,29.210000,16.002000,2.751840e+16
2,1970-12-31,-0.537097,-8.718871,14.264968,803.749871,NaN,NaN,18.216880,494.402452,203.044612,51.056988,3.015360e+16
3,1971-01-31,3.562742,-7.895323,6.677742,1092.049548,NaN,NaN,5.649310,762.260323,232.542976,73.754129,3.283200e+16
4,1971-02-28,4.261607,-8.661964,2.313214,1186.632214,NaN,NaN,1.164167,833.760000,181.676842,73.847158,3.538080e+16
...,...,...,...,...,...,...,...,...,...,...,...,...
583,2019-05-31,11.677419,-0.967742,5.645161,2038.645161,87.25,73.466667,1.870968,1203.290323,183.758065,102.513333,1.557965e+18
584,2019-06-30,19.833333,3.633333,0.233333,2148.366667,NaN,100.000000,0.000000,1239.500000,36.750000,15.929167,1.560600e+18
585,2019-07-31,22.935484,7.129032,0.000000,2150.000000,NaN,NaN,0.000000,1239.500000,0.000000,0.000000,1.563235e+18
586,2019-08-31,24.129032,7.935484,0.000000,2150.000000,NaN,100.000000,0.000000,1239.500000,0.000000,0.000000,1.565914e+18


In [ ]:
## SNSR data for that area 

In [ ]:
## saving code!!
def clip_ds_monthly(xds, basin, lon_name, var):
    # Adjust lon values to make sure they are within (-180, 180)
    xds['_longitude_adjusted'] = xr.where(
        xds[lon_name] > 180,
        xds[lon_name] - 360,
        xds[lon_name])

    # reassign the new coords to as the main lon coords
    # and sort DataArray using new coordinate values
    xds = (
        xds
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(xds._longitude_adjusted)})
        .drop(lon_name))

    xds = xds.rename({'_longitude_adjusted': lon_name})
    # xds = xds[[var]].transpose('ensyear','month', 'lat', 'lon')
    xds.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    xds.rio.write_crs("EPSG:4326", inplace=True)
    clipped_xds = xds.rio.clip(basin.geometry, all_touched = True, crs = basin.crs)

    return clipped_xds
    
def region_swei(clipped, basin, name = 'basin', plot = True, save = False):
    arr = clipped['swei'].data
    h1 = arr[:,:,0:165,:]
    h2 = arr[:,:,165:2*165,:]
    h3 = arr[:,:,2*165:3*165,:]
    h4 = arr[:,:,3*165:4*165,:]
    h5 = arr[:,:,4*165:5*165,:]
    hs = 5*165
    p1 = arr[:,:,hs:hs+86,:]
    p2 = arr[:,:,hs + 86:hs+2*86,:]
    p3 = arr[:,:,hs + 2*86:hs+3*86,:]
    p4 = arr[:,:,hs + 3*86:hs+4*86,:]
    p5 = arr[:,:,hs + 4*86:hs+5*86,:]
    alldata = [h1,h2,h3,h4,h5,p1,p2,p3,p4,p5]
    means = []
    for i in alldata:
        means.append(np.nanmean(i, axis=(0,1)).flatten())
    htime = range(165*12)
    ptime = range(165*12,165*12 + 85*12)
    hag = np.nanmean(means[0:5],axis=0)
    hag_sd = np.nanstd(means[0:5],axis=0)
    pag = np.nanmean(means[5:],axis=0)[0:85*12]
    pag_sd = np.nanstd(means[5:],axis=0)[0:85*12]
    ptime = ptime[0:85*12]
    if plot:
        fig,ax=plt.subplots()
        ax.plot(htime,hag)
        ax.fill_between(htime, hag-hag_sd, hag+hag_sd,alpha=0.5)
        ax.plot(ptime[0:85*12],pag[0:85*12])
        ax.fill_between(ptime, pag-pag_sd, pag+pag_sd,alpha=0.5)
        ax.set_title('SWEI for ' + name)
        plt.show()
    if save:
        np.save('/global/cscratch1/sd/cowherd/timeseries/swei_'+name+'.npy',arr)
        
##prep data
## load swei data
categ = np.load('/global/cscratch1/sd/cowherd/timeseries/categ.npy')
y = nc.Dataset('/global/cscratch1/sd/cowherd/timeseries/p1/monthly/H2OSNO_201501_210012.nc')['lat']
x = nc.Dataset('/global/cscratch1/sd/cowherd/timeseries/p1/monthly/H2OSNO_201501_210012.nc')['lon']
X,Y=np.meshgrid(x,y)

xds = xr.Dataset(
    data_vars=dict(
        swei=(["lat","lon","ensyear","month"],categ),
    ),
    coords=dict(
        lon=(["lon"], np.array(x)),
        lat=(["lat"], np.array(y)),
        ensyear=range(1255),
        month=range(12),
    ),
)

## specify basin
huc2fn = '/global/cscratch1/sd/cowherd/WBD_Annual_NRCS_OfficialSnapshot_ForTheCurrentFiscalYear/wbdhu2_a_us_september2021.gdb'
layers = fiona.listlayers(huc2fn)
for layer in layers:
    gdf = gpd.read_file(huc2fn,layer=layer)
caregion = gdf.loc[12]

import shapely
allhuc2 = [shapely.geometry.Polygon(gdf.loc[i].geometry[0]) for i in range(len(gdf))]
names = [gdf.loc[i]['name'] for i in range(len(gdf))]
huc2 = gpd.GeoDataFrame(data = {'name': names},
                                   geometry = allhuc2)
huc2 = huc2.set_crs('epsg:4326')

huc2.to_file('/global/cscratch1/sd/cowherd/WBD_Annual_NRCS_OfficialSnapshot_ForTheCurrentFiscalYear/huc2.shp')
cabasin = gpd.read_file('/global/cscratch1/sd/cowherd/WBD_Annual_NRCS_OfficialSnapshot_ForTheCurrentFiscalYear/cabasin.shp')


##repeat for HMA, or the world
basins = gpd.read_file('/global/homes/c/cowherd/data/hydrobasins/hybas_na/hybas_na_lev03_v1c.shp')
for i in range(len(basins)):
    basin = basins[basins.index == i]
    clipped = clip_ds_monthly(xds, basin = basin, lon_name = 'lon', var = 'swei')
    name = str(basin.HYBAS_ID[i])
    region_swei(clipped = clipped, basin = basin, plot=True, save = True, name = name)
fig,ax= plt.subplots()
changes = []
for i in range(len(basins)):
    basin = huc2[huc2.index == i]
    name = str(basin.HYBAS_ID[i])
    fn = '/global/cscratch1/sd/cowherd/timeseries/swei_'+name+'.npy'
    arr = np.load(fn)
    h1 = arr[:,:,0:165,:]
    h2 = arr[:,:,165:2*165,:]
    h3 = arr[:,:,2*165:3*165,:]
    h4 = arr[:,:,3*165:4*165,:]
    h5 = arr[:,:,4*165:5*165,:]
    hs = 5*165
    p1 = arr[:,:,hs:hs+86,:]
    p2 = arr[:,:,hs + 86:hs+2*86,:]
    p3 = arr[:,:,hs + 2*86:hs+3*86,:]
    p4 = arr[:,:,hs + 3*86:hs+4*86,:]
    p5 = arr[:,:,hs + 4*86:hs+5*86,:]
    alldata = [h1,h2,h3,h4,h5,p1,p2,p3,p4,p5]
    means = []
    for i in alldata:
        means.append(np.nanmean(i, axis=(0,1)).flatten())
    htime = range(165*12)
    ptime = range(165*12,165*12 + 85*12)
    hag = np.nanmean(means[0:5],axis=0)
    hag_sd = np.nanstd(means[0:5],axis=0)
    pag = np.nanmean(means[5:],axis=0)[0:85*12]
    pag_sd = np.nanstd(means[5:],axis=0)[0:85*12]
    ptime = ptime[0:85*12]
    change = np.nanmean(pag) - np.nanmean(hag)
    changes.append(change)

basins['change'] = changes
basins.plot(ax=ax,column = 'change', cmap='Reds_r',legend=True)
ax.set_xlim(-180,-50)
plt.show()